# Workshop demo 3
Create an interactive timeseries plot of the 10-min surface level BARRA sample data at a given location using the plotly package.

Nathan Eizenberg, Bureau of Meteorology, January 2018

## Setting up environment

Please refer to the instructions in the __[README.md](https://github.com/nathan-eize/AMOS2018-BARRA-data-demos/blob/master/README.md)__ for setting up the environment.

## Finding the BARRA sample data
This notebook uses ~100 netCDF files from the BARRA sample dataset, but will work with as many as you have in your ```dataDirRoot```, please define this accordingly. The full sample dataset is world readable (for the next few weeks only!) on the gdata1a filesystem at NCI, or you can download the files from our BOM public webpage.
### Access NCI
If you have any account to *any* project at NCI then you should be able to access the BARRA sample files - I have made the directories world readable for the AMOS workshop and for the next few weeks. Please find the sample data at the path ```/g/data/ma05/sample```. 
### Download via webpage
We have recently added the sample dataset to our __[BoM Reanlysis webpage](http://www.bom.gov.au/research/projects/reanalysis)__. You can download the files directly but please note that this host is not intended for bulk downloads. If you are using more than 100 files please consider getting an account on NCI to view the data in situ. 

## Running the notebook 
 2. Ensure you are in conda environment '*analysis3-unstable*' or equivalent 
 2. Choose which model from the sample dataset with the ```model``` variable:
     - *BARRA_R*, for the 12km, whole-of-Australia data assimilation system, or
     - *BARRA_TA*, for the 1.5km, Tasmania-only downscaling system
   
 
 2. Define your ```dataDirRoot``` to the top directory containing the sample data files with the filestructure defined in the __[sample data ReadMe file](http://www.bom.gov.au/clim_data/rrp/BARRA_sample/ReadMe)__.
 2. Run the entire notebook by choosing *Cell* --> *Run All*, from the Jupyter toolbar
 4. Scroll to the bottom to view the interactive plot in the ultimate cell
 5. Choose from the parameters on the left dropdown box to update the plot with the 24-hour timeseries of that parameter at the closest grid point of the given latitude and logitude coordinates
 6. Play around with the input configurations, choosing from 
     ```model```, ```latInput``` and ```lonInput``` 

In [ ]:
# Create an interactively adjustable time series of multiple BARRA parameters

## User configuration

In [ ]:
import os

In [ ]:
# specify which model of the sample data to plot
model = 'BARRA_R' # either 'BARRA_R' or 'BARRA_TA'
# define dirs of sample data
dataDirRoot = '/g/data/ma05/sample'
dataDirTemplate = os.path.join(dataDirRoot, '{model}/v1/{typeLong}/{stream}')
# lat lons of UNSW Quaderangle Lawn
latInput,lonInput = (-33.917089, 151.231058)

## Modules setup
We make handy use of the following great libraries:
 * pandas, making tabular data simple https://pandas.pydata.org/
 * iris, for dealing with gridded data of different formats developed by the UK MetOffice http://scitools.org.uk/iris/, and
 * plot.ly, for powerful interactive graphing classes https://plot.ly/python/

The plotly library is designed to create your plots online by default, however we will change this so that the plots are created only in this notebook. The ```plotly.offline``` methods copy the relevant plotly classes into the notebook and allow standalone, offline plotting. 

In [ ]:
from datetime import datetime
import pandas as pd

In [ ]:
import iris
import cf_units as units
iris.FUTURE.netcdf_promote = True

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# Built on plotly version 2.2.3 in the analysis3-unstable conda env
print(__version__)

In [ ]:
# plotly configuration to enable offline interactive plotting
init_notebook_mode(connected=True)

In [ ]:
# Set notebook tab completion to 'greedy'
%config IPCompleter.greedy=True

In [ ]:
# iso 1806 time format string
isoFormat = '%Y%m%dT%H%MZ'

## Define methods

In [ ]:
def get_ts_from_cube(cube, lonIdx,latIdx ):
    """
    Returns a single pixel timeseries iris cube given a full iris cube and a lat lon index pair
    """
    dimNameOrder = ['time','latitude','longitude']
    dimNames = [ dc.standard_name for dc in cube.dim_coords ]
    assert dimNameOrder == dimNames, "Unexpected dimension list/order {:}".format(dimNames)
    return cube[slice(cube.coord('forecast_period').points.tolist().index(6.0) + 1), latIdx, lonIdx]

In [ ]:
def return_ts_from_input_list( inSubList, latIn=latInput, lonIn=lonInput ):
    """
    Returns a dictionary of timeseries data from a list of netCDF consecutive filepaths of a single parameter
        and a given latitude and logitude (degrees)
    In:
        inSubList - a list of BARRA sample netCDF files, these should be of a single paramater
        latIn - float of latitude to choose closest North-South pixel, default global latInput
        lonIn - float of longitude to choose closest East-West pixel, default global lonInput
    Out:
        dict(
            tList - single list length N of datetime objects
            dList - single list length N of float values of param timeseries at pixel
            parStr - parameter string of cube
            parLongname - parameter long name of cube
            parUnit - iris cf unit object of physical units of parameter eg. m/s for windspeed
            parAttr - dictionary of all file metadata for that parameter
    """
    # get data
    cubes = iris.load(inSubList)
    # check that all parameters are the same
    assert all(c.standard_name == cubes[0].standard_name for c in cubes[1:]), "Not all cubes are same param"
    # get closest lat lon indices to input
    lonXIdx = cubes[0].coord(standard_name = 'longitude').nearest_neighbour_index(lonIn)
    latXIdx = cubes[0].coord(standard_name = 'latitude').nearest_neighbour_index(latIn)
    # return timeseries from each cube into list
    subCubes = [get_ts_from_cube(c, lonXIdx, latXIdx ) for c in cubes]
    # convert times to datetime objects and into a single list
    timeList = []
    [timeList.extend(c.coord('time').units.num2date(c.coord('time').points).tolist()) for c in subCubes];
    # and the data into a single list
    datList = []
    [datList.extend(c.data.tolist()) for c in subCubes];
    return dict(
        tList = timeList,
        dList = datList,
        parStr = cubes[0].standard_name,
        parLongname = cubes[0].long_name,
        parUnit = cubes[0].units,
        parAttr = cubes[0].attributes
    )

## Main script

### Locate files to plot

In [ ]:
# Scan through directory and create a dictionary of values:list_of_filepaths and key:parameter_string
sampleFileDict = {}
for root, dir, files in os.walk(dataDirTemplate.format(model=model, typeLong='forecast', stream='spec')):
    # want all surface level files in the spec dir that aren't on tiled dimensions or accumulations
    if 'accum' in root:
        continue
    if 'tiles' in root:
        continue
    if files == []:
        continue
    parStr = root.split('/')[-3] # grab the param
    fList = [os.path.join(root, f) for f in files if f.endswith('.nc')]
    fList.sort()
    sampleFileDict[parStr] = fList

In [ ]:
# this dictionary should be completely populated, you can limit the plot scope at this point by 
#    trimming the dictionary
sampleFileDict;


In [ ]:
print("Found {:d} params and a total of {:d} files".format(len(sampleFileDict.keys()), \
            len([file for p, flist in sampleFileDict.items() for file in flist ])))
print("List of unique params: \n\t{:}".format('\n\t'.join(set(sampleFileDict.keys()))))

### Pull out timeseries at given pixel and create data structures for plotting

In [ ]:
# Loop through sample data dict for each parameter, filter the single pixel timeseries
#   and store in a dictionary of pandas series', and metadata info
# THIS MAY TAKE 20-30 seconds to run!
seriesDict = {}
metaDict = {}
for param, fList in sampleFileDict.items():
    rDict = return_ts_from_input_list(fList)
    seriesDict[param] = pd.Series(data=rDict['dList'], index=rDict['tList'], name=rDict['parStr'])
    metaDict[param] = dict(
        units = rDict['parUnit'],
        attr = rDict['parAttr'],
        longname = rDict['parLongname']  
    )

In [ ]:
# And put series' into a single dataframe
df = pd.DataFrame.from_dict(seriesDict)

### Plot

In [ ]:
# A dictinary of data series for the plotly object, all the data are sitting in the single plot object,
# we use the plotly 'update' method to select which data are visible based on interactive selection
data = [go.Scatter(x=df.index, y=df[col], name=metaDict[col]['longname']) for col in df.columns]

In [ ]:
# The interactive plot object uses an 'update' dictionary to change the configuration of the plot layout.
# We create this dictionary in a loop for each of the dataframe columns, ie parameters. For each parameter
# 'button' we only want that parameter to be visible
buttonList = []
for col in df.columns:
    tmpDict = dict(
        label = col,
        method = 'update',
        args = [
            {'visible':[True if idx == df.columns.tolist().index(col) else False for idx in range(len(df.columns))]},
            {'title':'Timeseries of {} at location lat,lon:({:2.2f},{:2.2f})'.format(col, latInput, lonInput)},\
            {'yaxis':{'title':"{} [{}]".format(col, str(metaDict[col]['units'])), 'visible':True}}
        ]
    )
    buttonList.append(tmpDict)

updatemenus = [dict(buttons=buttonList)]

In [ ]:
# Define the main layout of the plot and enable the update methods defined earlier 
layout = go.Layout(
    title = 'Interactive timeseries plot for {} sample data at a chosen lat lon point'.format(model),
    showlegend = True,
    updatemenus=updatemenus
)

In [ ]:
fig = dict(data=data, layout=layout)

In [ ]:
iplot(fig)
# Choose the paramater from the dropdown box on the left